# Module 3: Data Acquisition

## Topic 1: Defining/Exploring Data Acquisition

## Topic 2: Getting/Storing Data

### For this topic, I'm going to show a couple of different ways to get csv files into your program

### You'll want to get comfortable with bringing data into your programs and manipulating it to the structure you need for analysis

### First I'll bring in a dataset and just output the data in my program

In [ ]:
import csv
with open('zoo dataset.csv') as csvfile:
    dataset_for_output = csv.reader(csvfile, delimiter=',')
    for row in dataset_for_output:
        print(row)

### This time, I'll exclude the first row because it is headers

In [ ]:
import csv
with open('zoo dataset.csv') as csvfile:
    dataset_for_output = csv.reader(csvfile, delimiter=',')
    row_count = 0
    for row in dataset_for_output:
        if row_count == 0:
            row_count +=1
        else:   
            print(row)
            row_count += 1
    print("total rows in file: ",row_count)

### Now I'll load it into a list of lists

In [ ]:
list_of_rows = []
import csv
with open('zoo dataset.csv') as csvfile:
    dataset_for_output = csv.reader(csvfile, delimiter=',')
    row_count = 0
    for row in dataset_for_output:
        if row_count == 0:
            row_count +=1
        else:   
            list_of_rows.append(row)
            row_count += 1
    print("total rows in file: ",row_count)
    print(list_of_rows)

### I could also get the data imported in a different structure using DictReader

In [ ]:
import csv
with open('zoo dataset.csv') as csvfile:
    dataset_for_output = csv.DictReader(csvfile, delimiter=',')
    print (dataset_for_output)
    for row in dataset_for_output:
        print(row)

### And because we will be doing a lot of stuff with Pandas this semester, we want to be able to directly load data into a dataframe

In [ ]:
import pandas as pd
with open('zoo dataset.csv') as csvfile:
    my_dataframe = pd.read_csv("zoo dataset.csv") 
my_dataframe.head()

## Topic 3: Web Scraper

### Web scraping is a good skill to build because it gives you a lot of opportunity to gather data from the entire web

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import csv

# specify the url
urlpage =  'https://www.fasttrack.co.uk/league-tables/tech-track-100/league-table/?leagueyear=2020'

# Structure the URL request to avoid 403 Forbidden Error
page = urllib.request.Request(urlpage, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'})

# query the website and return the html to the variable 'webpage'
webpage = urllib.request.urlopen(page).read()

# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(webpage, 'html.parser')
print(type(soup))
print(soup)

In [ ]:
import pandas as pd
list_of_dfs = pd.read_html(webpage,index_col=False)
print(type(list_of_dfs))
print(len(list_of_dfs))
print(list_of_dfs)
my_scrape_frame = pd.DataFrame(list_of_dfs[0])

In [ ]:
my_scrape_frame.head()


### Of course, we can do better.  We're data scientists and if there are 20 years worth of data readily available, we probably want them all.  Let's loop and grab everything :)

In [ ]:
import pandas as pd
base_url =  'https://www.fasttrack.co.uk/league-tables/tech-track-100/league-table/?leagueyear='
current_year = 2001
my_complete_scrape_frame = pd.DataFrame(columns = ['Rank','Company','Location','Year end','Annual sales rise over 3 years','Latest sales £000s', 'Staff', 'Comment','Year'])
while current_year <= 2020:
    current_url = base_url + str(current_year)
    print(current_url)
    current_page = urllib.request.Request(current_url, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'})
    current_webpage = urllib.request.urlopen(current_page).read()
    #print(current_webpage)
    current_list_of_dfs = pd.read_html(current_webpage,index_col=False)
    #print(current_list_of_dfs)
    my_current_scrape_frame = pd.DataFrame(current_list_of_dfs[0])
    #print(my_current_scrape_frame)
    my_current_scrape_frame['Year']=current_year
    print(my_current_scrape_frame)
    my_complete_scrape_frame = my_complete_scrape_frame.append(my_current_scrape_frame,ignore_index=True)
    print(my_complete_scrape_frame)
    current_year += 1

In [ ]:
print(my_complete_scrape_frame.head())
my_complete_scrape_frame.shape

## Topic 4: Database Integration

### This time I'm going to start with the easiest way to put this data into a database.  Because we ended up with a dataframe, we're going to have an easier time putting it into a database table

### I'm first creating a function that I can call to either create or connect to a sqlite database

In [ ]:
import sqlite3
from sqlite3 import Error

# This function creates a database connection and database if it doesn't already exist
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

### Now I can call my function to get my empty database created

In [ ]:
db_name = "tech_startups.db"
conn = create_connection(db_name)

### Now I'll quickly load my dataframe into my database

In [ ]:
my_complete_scrape_frame.to_sql("tech_data",conn,if_exists='replace',index=False)
conn.commit()
#c.close()
# I used if_exists=replace which drops an existing table and replaces it.
# I could have also used fail or append to either raise an error or add to existing table

### I did the easy way of loading first.  I can also use standard SQL to do most available DB integration options

### I'm going to add another table so I can load all of the company names and locations into it

In [ ]:
db_name = "tech_startups.db"
conn = create_connection(db_name)

sql_create_company_table = """CREATE TABLE IF NOT EXISTS Company (
                                    company_name text PRIMARY KEY,
                                    location text
                                );"""
if conn is not None:
    # create company table
    c = conn.cursor()
    c.execute(sql_create_company_table)
    conn.commit()
    c.close()

else:
    print("Error! cannot create the database connection.")

In [ ]:
company_list = my_complete_scrape_frame['Company'].tolist()
company_locations = my_complete_scrape_frame['Location'].tolist()
company_list_of_tuples = []
for item in range(len(company_list)):
    company_list_of_tuples.append((company_list[item],company_locations[item]))

In [ ]:
print(company_list[0:3])

In [ ]:
print(company_list_of_tuples[0:3])

In [ ]:
db_name = "tech_startups.db"
conn = create_connection(db_name)
#print(company_list_of_tuples)
sql_insert_company_table = "INSERT or REPLACE INTO Company (company_name, location) values (?,?)"
#if conn is not None:
    # insert company table
c = conn.cursor()
c.executemany(sql_insert_company_table, company_list_of_tuples)
conn.commit()
c.close()
#else:
    #print("Error! cannot create the database connection.")